# Omics Exercises

In [1]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
import cobra
import pandas as pd

from mewpy.omics import ExpressionSet
from mewpy.omics import GIMME

### Exercise 1

In [2]:
model = read_sbml_model('iMM904.xml')
model

Name,iMM904
Memory address,1383cae0828
Number of metabolites,1226
Number of reactions,1577
Number of genes,905
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


### Exercise 2

In [3]:
envcond = {'EX_glc__D_e':(-10,1000), "EX_xyl__D_e":(0,0), #glucose and xylose exchange
           "EX_epist_e":(-1000,1000), "EX_epistest_SC_e":(-1000,1000),"ERGSTt":(-1000,1000),
           "EX_ergstest_SC_e":(-1000,1000), "EX_fecost_e":(-1000,1000), "EX_lanost_e":(-1000,1000),  #sterols
           "EX_lanostest_SC_e":(-1000,1000), "EX_zymst_e":(-1000,1000), "EX_zymstest_SC_e":(-1000,1000),
           "ERGTETROLter":(-1000,1000), "EX_pc_SC_e":(-1000,1000), "EX_ptd1ino_SC_e":(-1000,1000),
           "EX_o2_e":(0,0) #oxygen exchange
          }

In [4]:
# COBRApy
for exchange in envcond.keys():
    drain = model.reactions.get_by_id(exchange)
    drain.bounds = envcond[exchange]
    print(drain.id, envcond[exchange])

EX_glc__D_e (-10, 1000)
EX_xyl__D_e (0, 0)
EX_epist_e (-1000, 1000)
EX_epistest_SC_e (-1000, 1000)
ERGSTt (-1000, 1000)
EX_ergstest_SC_e (-1000, 1000)
EX_fecost_e (-1000, 1000)
EX_lanost_e (-1000, 1000)
EX_lanostest_SC_e (-1000, 1000)
EX_zymst_e (-1000, 1000)
EX_zymstest_SC_e (-1000, 1000)
ERGTETROLter (-1000, 1000)
EX_pc_SC_e (-1000, 1000)
EX_ptd1ino_SC_e (-1000, 1000)
EX_o2_e (0, 0)


In [5]:
# MewPy
model_mewpy = read_sbml_model('iMM904.xml')
simul = get_simulator(model_mewpy, envcond = envcond)

### Exercise 3

In [6]:
pfba_solution = cobra.flux_analysis.pfba(model)
print(pfba_solution.fluxes["XYLR"])
print(pfba_solution.fluxes["XYLTD_D"])
print(pfba_solution.fluxes["XYLK"])
print(pfba_solution.fluxes["RPE"])
print(pfba_solution.fluxes["EX_etoh_e"])

0.0
0.0
0.0
-0.06176756734451793
16.7435803749062


In [7]:
# MewPy
res = simul.simulate(method="pFBA")
res.find(["XYLR","XYLTD_D","XYLK","RPE","EX_etoh_e"])

,Flux rate
Reaction ID,
EX_etoh_e,16.743580
RPE,-0.061768


### Exercise 4

In [8]:
envcond_xyl = {'EX_glc__D_e':(0,0), "EX_xyl__D_e":(-10,1000), #glucose and xylose exchange
               "EX_epist_e":(-1000,1000), "EX_epistest_SC_e":(-1000,1000),"ERGSTt":(-1000,1000),
               "EX_ergstest_SC_e":(-1000,1000), "EX_fecost_e":(-1000,1000), "EX_lanost_e":(-1000,1000),  #sterols
               "EX_lanostest_SC_e":(-1000,1000), "EX_zymst_e":(-1000,1000), "EX_zymstest_SC_e":(-1000,1000),
               "ERGTETROLter":(-1000,1000), "EX_pc_SC_e":(-1000,1000), "EX_ptd1ino_SC_e":(-1000,1000),
               "EX_o2_e":(0,0) #oxygen exchange
              }

In [9]:
for exchange in envcond_xyl.keys():
    drain = model.reactions.get_by_id(exchange)
    drain.bounds = envcond_xyl[exchange]

pfba_solution = cobra.flux_analysis.pfba(model)
print(pfba_solution.fluxes["XYLR"])
print(pfba_solution.fluxes["XYLTD_D"])
print(pfba_solution.fluxes["XYLK"])
print(pfba_solution.fluxes["RPE"])
print(pfba_solution.fluxes["EX_etoh_e"])

9.99999999999891
9.99999999999891
9.999999999998911
-3.3517552393830843
15.695453796022996


### Exercise 5

In [10]:
for exchange in envcond.keys():
    drain = model.reactions.get_by_id(exchange)
    drain.bounds = envcond[exchange]

essential_genes = cobra.flux_analysis.variability.find_essential_genes(model)
essential_reactions = cobra.flux_analysis.variability.find_essential_reactions(model)

In [11]:
len(essential_genes), len(essential_reactions)

(92, 156)

### Exercise 6

In [15]:
expression = pd.read_csv("gene_expression_data.txt",sep="\t",index_col=0)
expression.head(5)

,Glucose_medium,Xylose_medium
genes,,
YGR121C,8.75635,12.77670
YIL009W,31.34840,18.60180
YER089C,40.99820,28.90290
YML037C,5.24355,1.31501
YDR383C,7.88679,4.49923


In [20]:
expression['Glucose_medium']

genes
YGR121C      8.756350
YIL009W     31.348400
YER089C     40.998200
YML037C      5.243550
YDR383C      7.886790
              ...    
YDL114W      0.260572
YLR150W    629.845000
YOR255W      0.000000
YPL017C      5.928220
YMR278W     18.129100
Name: Glucose_medium, Length: 5939, dtype: float64

### Exercise 7

In [23]:
# Only on Mewpy, we will first use the glucose medium and then the xylose medium.
simul = get_simulator(model, envcond = envcond)

#get genes and expression data
genes = expression.index.values
conditions = ["Glucose_medium"]
exp = expression[conditions].to_numpy()

# load expression module
expr = ExpressionSet(genes, conditions, exp)

# Apply parsimonious GIMME in the model
g = GIMME(simul, expr, parsimonious= True)

# Fluxes results
g.find(["XYLR","XYLTD_D","XYLK","RPE","EX_etoh_e"])

expr

{'23CAPPD': 3.779729999999999, '2HMHMBQMTm': 5.097429999999999, '2HP6MPMOm': 5.097429999999999, '2HPMBQMTm': 5.097429999999999, '2HPMMBQMOm': 5.097429999999999, 'ACACT6p': 11.720428, 'ACACT7p': 11.720428, 'ACACT8p': 11.720428, 'ACACT9p': 11.720428, 'ACCOACrm': 6.629049999999999, 'AGTi': 10.81001, 'ACOAHim': 8.15859, 'ACOAO4p': 11.262046999999999, 'ACOAO5p': 11.262046999999999, 'ACOAO6p': 11.262046999999999, 'ACOAO7p': 11.262046999999999, 'ACOAO8p': 11.262046999999999, 'ACOAO9p': 11.262046999999999, 'ACOATAm': 7.145599999999999, 'ACON3MT': 0.15839999999999854, 'ACONT3m': 9.447569999999999, 'ACSm': 10.765659999999999, 'ACSp': 10.765659999999999, 'ACt2r': 9.515799999999999, 'ACtr': 11.729565, 'ADCS': 4.6807799999999995, 'ADK1m': 5.49203, 'ALDD19xr': 10.04297, 'ALDD20xm': 9.607859999999999, 'ALDD20y': 9.603919999999999, 'ALDD22x': 10.04297, 'ALDD2xm': 9.607859999999999, 'ALDD2y': 9.603919999999999, 'ADK3m': 5.49203, 'ADK4m': 5.49203, 'ADNCYC': 4.87135, 'ALLTAHr': 8.48039, 'ALLTN': 9.99551,

In [14]:
simul = get_simulator(model, envcond = envcond_xyl)

#get genes and expression data
genes = expression.index.values
conditions = ["Xylose_medium"]
exp = expression[conditions].to_numpy()

# load expression module
expr = ExpressionSet(genes, conditions, exp)

# Apply parsimonious GIMME in the model
g = GIMME(simul, expr, parsimonious=True)

# Fluxes results
g.find(["XYLR","XYLTD_D","XYLK","RPE","EX_etoh_e"])

{'23CAPPD': 2.461309999999999, '2HMHMBQMTm': 5.65515, '2HP6MPMOm': 5.65515, '2HPMBQMTm': 5.65515, '2HPMMBQMOm': 5.65515, 'ACACT6p': 10.012038, 'ACACT7p': 10.012038, 'ACACT8p': 10.012038, 'ACACT9p': 10.012038, 'ACCOAC': 0.6516099999999998, 'ACCOACrm': 8.91855, 'ACOAO4p': 9.43627, 'ACOAO5p': 9.43627, 'ACOAO6p': 9.43627, 'ACOAO7p': 9.43627, 'ACOAO8p': 9.43627, 'ACOAO9p': 9.43627, 'ACOATAm': 6.089239999999999, 'ACP1e': 2.26731, 'ACt2r': 9.47025, 'ACtr': 10.106266, 'ADCL': 5.8835299999999995, 'ADCS': 4.1363199999999996, 'ADHAPR_SC': 5.98891, 'ADK1m': 4.96474, 'ALDD20y': 9.39377, 'ALDD2y': 9.39377, 'ADK3m': 4.96474, 'ADK4m': 4.96474, 'ADNCYC': 5.831519999999999, 'ALLTAHr': 4.17078, 'ALLTN': 7.742929999999999, 'ALLTNti': 10.299094, 'ALLTTti': 3.14051, 'ALPHNH': 7.725429999999999, 'AMAOTr': 7.968439999999999, '3DH5HPBMTm': 5.65515, 'AMID': 4.45411, 'AMID2': 6.68665, 'AMID3': 6.68665, 'ATPtp_H': 1.854659999999999, 'AP4AH': 1.1367499999999993, 'BACCL': 4.2519, 'ARABR': 10.61375, 'ARGN': 7.13842,

,Flux rate
Reaction ID,
EX_etoh_e,14.375908
RPE,-3.066580
XYLK,9.150000
XYLR,9.150000
XYLTD_D,9.150000
